In [1]:
import json
from pathlib import Path

import pandas as pd

In [2]:
from bellem.dvc.experiment import load_experiments

filepaths = list(Path("../../tmp/musique-sweep-manual/").glob("*.json"))
experiments = [exp for fp in filepaths for exp in load_experiments(fp)]
print(f"{len(experiments)} experiments")
experiments[0]

16 experiments


{'commit': 'workspace',
 'id': 'workspace',
 'name': None,
 'params': {'dataset': {'path': 'bdsaglam/musique-sweep',
   'name': 'answerable',
   'split': 'train'},
  'qa': {'model': 'llama-3-70b-tgi',
   'temperature': 0.5,
   'system_prompt': 'ccot/format-thought.txt',
   'user_prompt_template': 'cq.txt',
   'few_shot_examples': 'manual/ccot.json',
   'n_shot': 2},
  'run': 1},
 'metrics': {'exact_match': 0.6533333333333333,
  'f1': 0.7498126503126502,
  'fuzzy_match': 0.7233333333333334,
  '2hops': {'exact_match': 0.74,
   'f1': 0.8350714285714287,
   'fuzzy_match': 0.82},
  '3hops': {'exact_match': 0.61,
   'f1': 0.7027950937950936,
   'fuzzy_match': 0.66},
  '4hops': {'exact_match': 0.61,
   'f1': 0.7115714285714286,
   'fuzzy_match': 0.69},
  'gen_token_count': {'all': {'count': 300.0,
    'mean': 46.49666666666667,
    'std': 13.946505138354052,
    'min': 16.0,
    '25%': 36.0,
    '50%': 45.0,
    '75%': 53.0,
    'max': 113.0},
   'success': {'count': 223.0,
    'mean': 44.533

In [3]:
df = pd.json_normalize(experiments).drop(columns=["commit"])
mask = (
    (df["params.dataset.path"] == "bdsaglam/musique-sweep")  &
    (df["params.qa.few_shot_examples"].str.contains("manual")) 
)

df = df.loc[mask].copy()
df.drop(columns=[col for col in df.columns if "fuzzy" in col], inplace=True)

param_cols = [col for col in df.columns if col.startswith("params.")]
metric_cols = [col for col in df.columns if col.startswith("metrics.")]
df.dropna(subset=param_cols, inplace=True, how="any")
df.drop_duplicates(subset=param_cols + metric_cols, inplace=True)

print(f"{len(df)} experiments after preprocessing")

12 experiments after preprocessing


In [4]:
def parse_qa_technique(system_prompt_filename):
    return system_prompt_filename.split("/")[0]

df["params.qa.technique"] = df["params.qa.system_prompt"].map(parse_qa_technique)

In [5]:
def sorted_tuple(x):
    return tuple(sorted(x))


run_counts = (
    df.groupby(
        [
            "params.qa.system_prompt",
            "params.qa.user_prompt_template",
            "params.qa.few_shot_examples",
            "params.qa.n_shot",
            "params.qa.temperature",
        ]
    )["params.run"]
    .aggregate(sorted_tuple)
    .reset_index()
)
run_counts.loc[run_counts["params.run"].map(len) != 3]

,params.qa.system_prompt,params.qa.user_prompt_template,params.qa.few_shot_examples,params.qa.n_shot,params.qa.temperature,params.run
0,ccot/format-thought.txt,cq.txt,manual/ccot.json,0,0.5,"(1,)"
1,ccot/format-thought.txt,cq.txt,manual/ccot.json,2,0.5,"(1,)"
2,cot/format-thought.txt,cq.txt,manual/cot.json,0,0.5,"(1,)"
3,cot/format-thought.txt,cq.txt,manual/cot.json,1,0.5,"(1,)"
4,cot/format-thought.txt,cq.txt,manual/cot.json,2,0.5,"(1,)"
5,cot/original.txt,cq.txt,manual/cot-original.json,1,0.5,"(1,)"
6,cot/original.txt,cq.txt,manual/cot-original.json,2,0.5,"(1,)"
7,cte/format-triplets-few.txt,cq.txt,manual/cte.json,2,0.5,"(1,)"
8,direct/format-few.txt,cq.txt,manual/direct.json,0,0.5,"(1, 1)"
9,direct/format-few.txt,cq.txt,manual/direct.json,1,0.5,"(1,)"


In [6]:
for col in run_counts.columns:
    print(f"- {col}")
    for value in run_counts[col].unique():
        print(f"\t{value}")

- params.qa.system_prompt
	ccot/format-thought.txt
	cot/format-thought.txt
	cot/original.txt
	cte/format-triplets-few.txt
	direct/format-few.txt
- params.qa.user_prompt_template
	cq.txt
- params.qa.few_shot_examples
	manual/ccot.json
	manual/cot.json
	manual/cot-original.json
	manual/cte.json
	manual/direct.json
- params.qa.n_shot
	0
	2
	1
- params.qa.temperature
	0.5
- params.run
	(1,)
	(1, 1)


In [7]:
df.to_json('exps.jsonl', orient='records', lines=True)